In [4]:
# 기본 설정


from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from matplotlib import pyplot as plt
import pandas as pd


# confusion_matrix
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    # 사이즈 변경: figsize = (x, y) -> x, y로 사이즈를 변경해 주세요
    fig, ax = plt.subplots(figsize=(30, 30)) 
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.show()


# 설정에 따라 수정이 필요합니다
# path -> 분석하고 싶은 inference_file을 설정해 주세요
# ex) log/{inference_file}/ ... 
# ex) prediction/{inference_file}/ ...
path = '2023-05-13-22:40:34' # 이 부분만 변경하시면 됩니다.


model_info = f'/opt/ml/level2_klue-nlp-01/code/logs/{path}/best_model/config.json'
no_preprocess_train_path = f'/opt/ml/level2_klue-nlp-01/code/logs/{path}/split_data/split_nopreprocess_train.csv'
preprocess_train_path = f'/opt/ml/level2_klue-nlp-01/code/logs/{path}/split_data/split_preprocess_train.csv'
val_inference_path = f'/opt/ml/level2_klue-nlp-01/code/prediction/{path}/val_inference.csv'
no_preprocess_val_path = f'/opt/ml/level2_klue-nlp-01/code/logs/{path}/split_data/split_nopreprocess_val.csv'
preprocess_val_path = f'/opt/ml/level2_klue-nlp-01/code/logs/{path}/split_data/split_preprocess_val.csv'

no_preprocess_train_df = pd.read_csv(no_preprocess_train_path)
preprocess_train_df = pd.read_csv(preprocess_train_path)
val_inference_df = pd.read_csv(val_inference_path)
no_preprocess_val_df = pd.read_csv(no_preprocess_val_path)
preprocess_val_df = pd.read_csv(preprocess_val_path)

# 틀린 데이터 추출
wrong_df = val_inference_df[val_inference_df['label'] != val_inference_df['pred_label']]
print(val_inference_df.columns.values)
print("wrong data >>", len(wrong_df))


['id' 'sentence' 'subject_entity' 'object_entity' 'label' 'pred_label'
 'probs']
wrong data >> 1370


In [6]:
# confusion_matrix 출력

label = val_inference_df['label']
pred_label = val_inference_df['pred_label']
# plot_confusion_matrix(pred_label, label, label.unique())


In [23]:
# 데이터 분석 (갯수, 비율)
# 필요한 부분만 주석 해제해서 사용해 주세요


# print(">>> val 데이터 갯수")
# print(val_inference_df['label'].value_counts())
# val_inference_df['label'].value_counts().plot(kind='barh', figsize=(30,30))
# plt.show()
# print()


# print(">>> 틀린 데이터 갯수")
# print(wrong_df['label'].value_counts())
# wrong_df['label'].value_counts().plot(kind='barh', figsize=(30,30))
# plt.show()
# print()


# print(">>> 틀린 데이터 비율")
# print(wrong_df['label'].value_counts(normalize=True))
# wrong_df['label'].value_counts(normalize=True).plot(kind='barh', figsize=(30,30))
# plt.show()
# print()


# print(">>> val 데이터 대비, 틀린 데이터 비율")
# ratio_list = []
# label = wrong_df['label'].unique()
# for i in range(len(label)):
#     count = len(wrong_df[wrong_df['label'] == label[i]]) / len(no_preprocess_train_df[no_preprocess_train_df['label'] == label[i]])
#     ratio_list.append(list(["{0:38}".format(label[i]), round(count,3)]))
# ratio_list = sorted(ratio_list, key=lambda x:x[1], reverse=True)
# for i in range(len(label)):
#     print(ratio_list[i][0],ratio_list[i][1])


In [76]:
label = val_inference_df['label']
pred_label = val_inference_df['pred_label']

# print(">>> label 정보")
# print(label.unique())
# print()

# 데이터 분석 (detail)
# 필요한 부분만 주석 해제해서 사용해 주세요
# selected_label -> 분석을 원하는 라벨로 변경해 주세요
selected_label = 'org:top_members/employees'

selected_df = val_inference_df[val_inference_df['label'] == selected_label]
selected_wrong_df = selected_df[selected_df['label'] != selected_df['pred_label']]

# print(f">>> 예측한 {selected_label} 데이터 갯수")
# print(selected_df['label'].value_counts())
# print()
# print(f">>> 예측한 {selected_label} 중, 틀린 데이터")
# print(selected_wrong_df['pred_label'].value_counts())
# selected_wrong_df['pred_label'].value_counts().plot(kind='barh')
# plt.show()
# print()

# print(">>> 예측한 {selected_label} 중, 틀린 데이터 정보")
# print()
# for i in selected_wrong_df.index:
#     print(f"index >>> ",i)
#     print(f"{selected_wrong_df['sentence'][i]}")
#     print(f"subject >>> {selected_wrong_df['subject_entity'][i]}")
#     print(f"object >>> {selected_wrong_df['object_entity'][i]}")
#     print(f"pred_relation >>> {selected_wrong_df['pred_label'][i]}")
#     print()
